# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [8]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [9]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [10]:
# 3) seu código aqui
colunas_selecionadas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
selecao = sinasc[colunas_selecionadas]
valores_ausentes_por_coluna = selecao.isna().sum()
print(valores_ausentes_por_coluna)

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64


In [11]:
# 4) seu código aqui
selecao = selecao.dropna(subset=['APGAR5'])
num_linhas_apos_remocao = selecao.shape[0]
valores_ausentes_apos_remocao = selecao.isna().sum()
print(f"Número de linhas após remoção: {num_linhas_apos_remocao}")
print("\nValores ausentes por coluna após remoção:")
print(valores_ausentes_apos_remocao)

Número de linhas após remoção: 26925

Valores ausentes por coluna após remoção:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


In [12]:
# 5) seu código aqui
selecao['ESTCIVMAE'].replace(9, pd.NA, inplace=True)
selecao['CONSULTAS'].replace(9, pd.NA, inplace=True)
selecao = selecao.dropna(subset=['APGAR5', 'ESTCIVMAE', 'CONSULTAS'])
num_linhas_apos_remocao = selecao.shape[0]
valores_ausentes_apos_remocao = selecao.isna().sum()
print(f"Número de linhas após remoção: {num_linhas_apos_remocao}")
print("\nValores ausentes por coluna após remoção:")
print(valores_ausentes_apos_remocao)

Número de linhas após remoção: 26416

Valores ausentes por coluna após remoção:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         242
QTDFILVIVO    1525
GESTACAO      1174
GRAVIDEZ        73
CONSULTAS        0
APGAR5           0
dtype: int64


In [13]:
# 6) Seu código aqui
selecao['QTDFILVIVO'].fillna(0, inplace=True)
num_linhas_apos_substituicao = selecao.shape[0]
valores_ausentes_apos_substituicao = selecao.isna().sum()
print(f"Número de linhas após substituição: {num_linhas_apos_substituicao}")
print("\nValores ausentes por coluna após substituição:")
print(valores_ausentes_apos_substituicao)

Número de linhas após substituição: 26416

Valores ausentes por coluna após substituição:
LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         242
QTDFILVIVO       0
GESTACAO      1174
GRAVIDEZ        73
CONSULTAS        0
APGAR5           0
dtype: int64


In [14]:
# 7) seu código aqui
colunas_qualitativas = ['ESTCIVMAE', 'GESTACAO', 'GRAVIDEZ']

for coluna in colunas_qualitativas:
    moda_coluna = selecao[coluna].mode().iloc[0]  # Moda
    selecao[coluna].fillna(moda_coluna, inplace=True)

num_linhas_apos_substituicao = selecao.shape[0]
valores_ausentes_apos_substituicao = selecao.isna().sum()

print(f"Número de linhas após substituição: {num_linhas_apos_substituicao}")
print("\nValores ausentes por coluna após substituição:")
print(valores_ausentes_apos_substituicao)

#Fiz uma seleção das colunas qualitativas e optei pela moda, visto que não são muitas classificações e a chance de acertar fica
#maior

Número de linhas após substituição: 26416

Valores ausentes por coluna após substituição:
LOCNASC         0
IDADEMAE        0
ESTCIVMAE       0
ESCMAE        242
QTDFILVIVO      0
GESTACAO        0
GRAVIDEZ        0
CONSULTAS       0
APGAR5          0
dtype: int64


In [15]:
# 8) seu código aqui
selecao['APGAR_CATEGORIA'] = pd.cut(selecao['APGAR5'], bins=[0, 3, 5, 7, 10], labels=['asfixia severa', 'asfixia moderada', 'asfixia leve', 'normal'], include_lowest=True)

frequencias_apgar_categoria = selecao['APGAR_CATEGORIA'].value_counts()

print("Frequências da categorização do Apgar:")
print(frequencias_apgar_categoria)

Frequências da categorização do Apgar:
APGAR_CATEGORIA
normal              25964
asfixia leve          312
asfixia severa         72
asfixia moderada       68
Name: count, dtype: int64


In [16]:
# 9) seu código aqui
selecao.rename(columns={
    'LOCNASC': 'loc_nasc',
    'IDADEMAE': 'idade_mae',
    'ESTCIVMAE': 'est_civ_mae',
    'ESCMAE': 'esc_mae',
    'QTDFILVIVO': 'qtd_fil_vivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5',
    'APGAR_CATEGORIA': 'apgar_categoria'
}, inplace=True)

print(selecao.head())

   loc_nasc  idade_mae est_civ_mae          esc_mae  qtd_fil_vivo  \
0         1         19         5.0      8 a 11 anos           0.0   
1         1         29         2.0      8 a 11 anos           1.0   
3         1         30         5.0  12 anos ou mais           0.0   
4         1         30         2.0      8 a 11 anos           1.0   
5         1         33         5.0       1 a 3 anos           2.0   

          gestacao gravidez consultas  apgar5 apgar_categoria  
0  37 a 41 semanas    Única         4    10.0          normal  
1  37 a 41 semanas    Única         4     9.0          normal  
3  37 a 41 semanas    Única         3    10.0          normal  
4  37 a 41 semanas    Única         4    10.0          normal  
5  37 a 41 semanas    Única         4    10.0          normal  
